In [1]:
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [3]:
#savant_df2016 = statcast(start_dt = "2016-03-28", end_dt = "2016-11-15")

In [4]:
#savant_df2017 = statcast(start_dt = "2017-03-28", end_dt = "2017-11-15")

In [5]:
#savant_df2018 = statcast(start_dt = "2018-03-28", end_dt = "2018-11-15")

In [6]:
#savant_df2021 = statcast(start_dt = "2021-03-28", end_dt = "2021-11-15")

In [7]:
#savant_df2022 = statcast(start_dt = "2022-03-28", end_dt = "2022-11-15")

In [8]:
#expected = pd.concat([savant_df2016, savant_df2017, savant_df2018, savant_df2021, savant_df2022], axis = 0)

In [9]:
expected = pd.read_csv("Expected.csv")

#pd.set_option('display.max_columns', None)

expected

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_37879/1777417083.py:1: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  expected = pd.read_csv("Expected.csv")


,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,"Michael Martinez grounds out softly, third bas...",W,R,L,CLE,CHC,X,5.0,ground_ball,0,1,2016,-0.78,-1.32,0.75,2.85,NaN,NaN,434658.0,2,10,Bot,112.98,177.44,NaN,NaN,471083,NaN,161103_005034,-2.971400,-112.375382,3.212320,-5.837,20.536,-43.549,3.37,1.56,2.0,70.4,-53.0,76.9,2324.0,6.2,487637.0,543557.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.113,0.102,0.0,1.0,0.0,0.0,2.0,89.0,2.0,Curveball,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
1,CU,2016-11-02,74.3,3.04,6.51,Mike Montgomery,492841,543557,NaN,called_strike,NaN,NaN,NaN,NaN,3.0,"Michael Martinez grounds out softly, third bas...",W,R,L,CLE,CHC,S,NaN,NaN,0,0,2016,-0.68,-1.29,0.43,2.64,NaN,NaN,434658.0,2,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_005016,-3.966784,-109.018497,3.286612,-4.543,19.370,-42.641,3.08,1.34,NaN,NaN,NaN,74.3,2478.0,6.0,487637.0,543557.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,1.0,Curveball,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,318.0,0.000,-0.040,-15.48,-8.16,8.16,3.04
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,NaN,NaN,NaN,NaN,8.0,Rajai Davis singles on a line drive to center ...,W,R,R,CLE,CHC,X,8.0,line_drive,1,0,2016,0.04,1.46,0.04,1.56,NaN,446386.0,NaN,2,10,Bot,122.11,107.98,NaN,NaN,471083,NaN,161103_004639,3.385197,-138.657741,-9.902501,-0.221,29.317,-11.776,3.30,1.48,254.0,98.7,9.0,96.3,2636.0,6.8,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.687,0.618,0.9,1.0,1.0,0.0,4.0,88.0,2.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
3,FF,2016-11-02,95.6,-1.28,6.51,Carl Edwards Jr.,434658,605218,NaN,ball,NaN,NaN,NaN,NaN,14.0,Rajai Davis singles on a line drive to center ...,W,R,R,CLE,CHC,B,NaN,NaN,0,0,2016,-0.18,1.47,0.40,1.04,NaN,NaN,446386.0,2,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004618,4.883507,-139.407469,-11.639133,-3.197,25.774,-11.146,2.95,1.38,NaN,NaN,NaN,97.2,2672.0,6.8,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,1.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard,191.0,0.002,0.031,17.64,-2.16,2.16,1.28
4,FF,2016-11-02,94.8,-1.13,6.30,Carl Edwards Jr.,446386,605218,walk,ball,NaN,NaN,NaN,NaN,14.0,Brandon Guyer walks.,W,R,R,CLE,CHC,B,NaN,NaN,3,1,2016,0.23,1.21,0.16,0.78,NaN,NaN,NaN,2,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004459,2.757214,-138.305091,-10.905638,2.476,23.633,-15.029,3.25,1.51,NaN,NaN,NaN,96.8,2712.0,6.7,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,0.7,1.0,0.0,0.0,NaN,87.0,5.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard

In [10]:
# Converts the Induced Vertical Break and Horizontal Break to inches from feet
expected['IVB'] = expected['pfx_z'] * 12
expected['HB'] = expected['pfx_x'] * 12
expected['ABS_HB'] = np.abs(expected["HB"])
expected['ABS_RelSide'] = np.abs(expected["release_pos_x"])

expected = expected.rename(columns={'player_name': 'Name', "pitch_type" : "Pitch Type", "release_pos_z" : "RelHeight"})
expected

,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,"Michael Martinez grounds out softly, third bas...",W,R,L,CLE,CHC,X,5.0,ground_ball,0,1,2016,-0.78,-1.32,0.75,2.85,NaN,NaN,434658.0,2,10,Bot,112.98,177.44,NaN,NaN,471083,NaN,161103_005034,-2.971400,-112.375382,3.212320,-5.837,20.536,-43.549,3.37,1.56,2.0,70.4,-53.0,76.9,2324.0,6.2,487637.0,543557.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.113,0.102,0.0,1.0,0.0,0.0,2.0,89.0,2.0,Curveball,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
1,CU,2016-11-02,74.3,3.04,6.51,Mike Montgomery,492841,543557,NaN,called_strike,NaN,NaN,NaN,NaN,3.0,"Michael Martinez grounds out softly, third bas...",W,R,L,CLE,CHC,S,NaN,NaN,0,0,2016,-0.68,-1.29,0.43,2.64,NaN,NaN,434658.0,2,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_005016,-3.966784,-109.018497,3.286612,-4.543,19.370,-42.641,3.08,1.34,NaN,NaN,NaN,74.3,2478.0,6.0,487637.0,543557.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,1.0,Curveball,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,318.0,0.000,-0.040,-15.48,-8.16,8.16,3.04
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,NaN,NaN,NaN,NaN,8.0,Rajai Davis singles on a line drive to center ...,W,R,R,CLE,CHC,X,8.0,line_drive,1,0,2016,0.04,1.46,0.04,1.56,NaN,446386.0,NaN,2,10,Bot,122.11,107.98,NaN,NaN,471083,NaN,161103_004639,3.385197,-138.657741,-9.902501,-0.221,29.317,-11.776,3.30,1.48,254.0,98.7,9.0,96.3,2636.0,6.8,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.687,0.618,0.9,1.0,1.0,0.0,4.0,88.0,2.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
3,FF,2016-11-02,95.6,-1.28,6.51,Carl Edwards Jr.,434658,605218,NaN,ball,NaN,NaN,NaN,NaN,14.0,Rajai Davis singles on a line drive to center ...,W,R,R,CLE,CHC,B,NaN,NaN,0,0,2016,-0.18,1.47,0.40,1.04,NaN,NaN,446386.0,2,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004618,4.883507,-139.407469,-11.639133,-3.197,25.774,-11.146,2.95,1.38,NaN,NaN,NaN,97.2,2672.0,6.8,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,1.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard,191.0,0.002,0.031,17.64,-2.16,2.16,1.28
4,FF,2016-11-02,94.8,-1.13,6.30,Carl Edwards Jr.,446386,605218,walk,ball,NaN,NaN,NaN,NaN,14.0,Brandon Guyer walks.,W,R,R,CLE,CHC,B,NaN,NaN,3,1,2016,0.23,1.21,0.16,0.78,NaN,NaN,NaN,2,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004459,2.757214,-138.305091,-10.905638,2.476,23.633,-15.029,3.25,1.51,NaN,NaN,NaN,96.8,2712.0,6.7,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,0.7,1.0,0.0,0.0,NaN,87.0,5.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard

In [11]:
df_swing = expected[expected['description'].isin(['swinging_strike', 'swinging_strike_blocked', 'hit_into_play', 'foul'])]
df_swing

,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,"Michael Martinez grounds out softly, third bas...",W,R,L,CLE,CHC,X,5.0,ground_ball,0,1,2016,-0.78,-1.32,0.75,2.85,NaN,NaN,434658.0,2,10,Bot,112.98,177.44,NaN,NaN,471083,NaN,161103_005034,-2.971400,-112.375382,3.212320,-5.837,20.536,-43.549,3.37,1.56,2.0,70.4,-53.0,76.9,2324.0,6.2,487637.0,543557.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.113,0.102,0.0,1.0,0.0,0.0,2.0,89.0,2.0,Curveball,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,NaN,NaN,NaN,NaN,8.0,Rajai Davis singles on a line drive to center ...,W,R,R,CLE,CHC,X,8.0,line_drive,1,0,2016,0.04,1.46,0.04,1.56,NaN,446386.0,NaN,2,10,Bot,122.11,107.98,NaN,NaN,471083,NaN,161103_004639,3.385197,-138.657741,-9.902501,-0.221,29.317,-11.776,3.30,1.48,254.0,98.7,9.0,96.3,2636.0,6.8,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.687,0.618,0.9,1.0,1.0,0.0,4.0,88.0,2.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
9,CU,2016-11-02,81.3,-0.98,6.39,Carl Edwards Jr.,608070,605218,field_out,hit_into_play,NaN,NaN,NaN,NaN,13.0,"Jose Ramirez grounds out, shortstop Addison Ru...",W,L,R,CLE,CHC,X,6.0,ground_ball,2,2,2016,1.16,-0.99,-1.20,2.10,NaN,NaN,NaN,1,10,Bot,117.04,147.02,NaN,NaN,471083,NaN,161103_004229,-3.104547,-119.195288,0.446945,11.310,22.889,-41.354,3.34,1.51,8.0,86.1,-13.0,82.0,2468.0,6.4,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.100,0.090,0.0,1.0,0.0,0.0,2.0,86.0,5.0,Curveball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard,44.0,-0.029,-0.144,-11.88,13.92,13.92,0.98
12,CU,2016-11-02,82.4,-1.17,6.30,Carl Edwards Jr.,608070,605218,NaN,foul,NaN,NaN,NaN,NaN,9.0,"Jose Ramirez grounds out, shortstop Addison Ru...",W,L,R,CLE,CHC,S,NaN,NaN,0,1,2016,1.08,-1.06,0.47,1.54,NaN,NaN,NaN,1,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004112,1.285163,-120.747317,-0.691823,9.969,22.736,-42.064,3.34,1.51,177.0,73.5,17.0,81.7,2695.0,5.4,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,2.0,Curveball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard,51.0,0.000,-0.037,-12.72,12.96,12.96,1.17
14,FF,2016-11-02,95.2,-1.16,6.37,Carl Edwards Jr.,435063,605218,strikeout,swinging_strike,NaN,NaN,NaN,NaN,14.0,Mike Napoli strikes out swinging.,W,R,R,CLE,CHC,S,2.0,NaN,2,2,2016,0.44,1.17,1.71,2.10,NaN,NaN,NaN,0,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004013,6.367670,-139.168623,-7.582581,4.264,28.193,-15.826,3.23,1.44,NaN,NaN,NaN,97.0,2703.0,6.9,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,0.0,1.0,0.0,0.0,NaN,85.0,5.0,4-Seam

In [12]:
df_swing['Whiff'] = df_swing['description'].apply(lambda x: 1 if x == 'swinging_strike' else 0)
df_swing

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_37879/3217903474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swing['Whiff'] = df_swing['description'].apply(lambda x: 1 if x == 'swinging_strike' else 0)


,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide,Whiff
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,"Michael Martinez grounds out softly, third bas...",W,R,L,CLE,CHC,X,5.0,ground_ball,0,1,2016,-0.78,-1.32,0.75,2.85,NaN,NaN,434658.0,2,10,Bot,112.98,177.44,NaN,NaN,471083,NaN,161103_005034,-2.971400,-112.375382,3.212320,-5.837,20.536,-43.549,3.37,1.56,2.0,70.4,-53.0,76.9,2324.0,6.2,487637.0,543557.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.113,0.102,0.0,1.0,0.0,0.0,2.0,89.0,2.0,Curveball,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93,0
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,NaN,NaN,NaN,NaN,8.0,Rajai Davis singles on a line drive to center ...,W,R,R,CLE,CHC,X,8.0,line_drive,1,0,2016,0.04,1.46,0.04,1.56,NaN,446386.0,NaN,2,10,Bot,122.11,107.98,NaN,NaN,471083,NaN,161103_004639,3.385197,-138.657741,-9.902501,-0.221,29.317,-11.776,3.30,1.48,254.0,98.7,9.0,96.3,2636.0,6.8,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.687,0.618,0.9,1.0,1.0,0.0,4.0,88.0,2.0,4-Seam Fastball,6.0,8.0,6.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29,0
9,CU,2016-11-02,81.3,-0.98,6.39,Carl Edwards Jr.,608070,605218,field_out,hit_into_play,NaN,NaN,NaN,NaN,13.0,"Jose Ramirez grounds out, shortstop Addison Ru...",W,L,R,CLE,CHC,X,6.0,ground_ball,2,2,2016,1.16,-0.99,-1.20,2.10,NaN,NaN,NaN,1,10,Bot,117.04,147.02,NaN,NaN,471083,NaN,161103_004229,-3.104547,-119.195288,0.446945,11.310,22.889,-41.354,3.34,1.51,8.0,86.1,-13.0,82.0,2468.0,6.4,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,0.100,0.090,0.0,1.0,0.0,0.0,2.0,86.0,5.0,Curveball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard,44.0,-0.029,-0.144,-11.88,13.92,13.92,0.98,0
12,CU,2016-11-02,82.4,-1.17,6.30,Carl Edwards Jr.,608070,605218,NaN,foul,NaN,NaN,NaN,NaN,9.0,"Jose Ramirez grounds out, shortstop Addison Ru...",W,L,R,CLE,CHC,S,NaN,NaN,0,1,2016,1.08,-1.06,0.47,1.54,NaN,NaN,NaN,1,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004112,1.285163,-120.747317,-0.691823,9.969,22.736,-42.064,3.34,1.51,177.0,73.5,17.0,81.7,2695.0,5.4,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,2.0,Curveball,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard,51.0,0.000,-0.037,-12.72,12.96,12.96,1.17,0
14,FF,2016-11-02,95.2,-1.16,6.37,Carl Edwards Jr.,435063,605218,strikeout,swinging_strike,NaN,NaN,NaN,NaN,14.0,Mike Napoli strikes out swinging.,W,R,R,CLE,CHC,S,2.0,NaN,2,2,2016,0.44,1.17,1.71,2.10,NaN,NaN,NaN,0,10,Bot,NaN,NaN,NaN,NaN,471083,NaN,161103_004013,6.367670,-139.168623,-7.582581,4.264,28.193,-15.826,3.23,1.44,NaN,NaN,NaN,97.0,2703.0,6.9,487637.0,605218.0,471083.0,519203.0,595879.0,592178.0,608365.0,450314.0,451594.0,518792.0,54.5,NaN,NaN,0.0,1.0,0.0,0.0,NaN,8

In [13]:
df_swing = df_swing[["Name", "Pitch Type", "release_speed", "release_spin_rate", "IVB", "ABS_HB", "RelHeight", "ABS_RelSide", "release_extension", "Whiff"]].dropna()
df_swing

,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension,Whiff
0,Mike Montgomery,CU,76.6,2324.0,-15.84,9.36,6.59,2.93,6.2,0
2,Carl Edwards Jr.,FF,94.9,2636.0,17.52,0.48,6.46,1.29,6.8,0
9,Carl Edwards Jr.,CU,81.3,2468.0,-11.88,13.92,6.39,0.98,6.4,0
12,Carl Edwards Jr.,CU,82.4,2695.0,-12.72,12.96,6.30,1.17,5.4,0
14,Carl Edwards Jr.,FF,95.2,2703.0,14.04,5.28,6.37,1.16,6.9,1
...,...,...,...,...,...,...,...,...,...,...
3658515,Adam Wainwright,FC,84.0,2349.0,8.88,6.24,6.32,1.26,6.4,0
3658518,Adam Wainwright,SI,88.4,2063.0,12.84,14.04,6.21,1.13,6.5,0
3658519,Adam Wainwright,CH,81.9,1610.0,5.04,10.56,6.26,1.34,6.2,1
3658520,Adam Wainwright,SI,87.7,2088.0,9.60,11.52,6.26,1.29,6.4,0


In [14]:
df_sinker = df_swing[(df_swing["Pitch Type"] == "SI")]
df_cutter = df_swing[df_swing["Pitch Type"] == "FC"]
df_fastball = df_swing[df_swing["Pitch Type"] == "FF"]
df_changeup = df_swing[df_swing["Pitch Type"] == "CH"]
df_splitter = df_swing[(df_swing["Pitch Type"] == "FS") | (df_swing["Pitch Type"] == "FO")]
df_curveball = df_swing[(df_swing["Pitch Type"] == "CU") | (df_swing["Pitch Type"] == "KC")]
df_slider = df_swing[(df_swing["Pitch Type"] == "SL") | (df_swing["Pitch Type"] == "SV")]
df_sweeper = df_swing[df_swing["Pitch Type"] == "ST"]

## Sinker

In [15]:
SIfeature = df_sinker.drop(["Name", "Pitch Type", "Whiff"], axis=1)
SIlabel = df_sinker['Whiff']

SIrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(SIfeature, SIlabel)

## Cutter

In [16]:
CTfeature = df_cutter.drop(["Name", "Pitch Type", "Whiff"], axis=1)
CTlabel = df_cutter['Whiff']

CTrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(CTfeature, CTlabel)

## 4-Seam

In [17]:
FBfeature = df_fastball.drop(["Name", "Pitch Type", "Whiff"], axis=1)
FBlabel = df_fastball['Whiff']

FBrf = RandomForestRegressor(n_estimators = 30, random_state = 42).fit(FBfeature, FBlabel)

## Changeup

In [18]:
CHfeature = df_changeup.drop(["Name", "Pitch Type", "Whiff"], axis=1)
CHlabel = df_changeup['Whiff']

CHrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(CHfeature, CHlabel)

## Splitter

In [19]:
SPfeature = df_splitter.drop(["Name", "Pitch Type", "Whiff"], axis=1)
SPlabel = df_splitter['Whiff']

SPrf = RandomForestRegressor(n_estimators = 10, random_state = 42).fit(SPfeature, SPlabel)

## Curveball

In [20]:
CUfeature = df_curveball.drop(["Name", "Pitch Type", "Whiff"], axis=1)
CUlabel = df_curveball['Whiff']

CUrf = RandomForestRegressor(n_estimators = 20, random_state = 42).fit(CUfeature, CUlabel)

## Slider

In [21]:
SLfeature = df_slider.drop(["Name", "Pitch Type", "Whiff"], axis=1)
SLlabel = df_slider['Whiff']

SLrf = RandomForestRegressor(n_estimators = 25, random_state = 42).fit(SLfeature, SLlabel)

## Sweeper

In [22]:
STfeature = df_sweeper.drop(["Name", "Pitch Type", "Whiff"], axis=1)
STlabel = df_sweeper['Whiff']

STrf = RandomForestRegressor(n_estimators = 10, random_state = 42).fit(STfeature, STlabel)

## 2023 DF

In [70]:
savant_df = statcast(start_dt = "2023-03-30", end_dt = "2023-10-02")

savant_df['IVB'] = savant_df['pfx_z'] * 12
savant_df['HB'] = savant_df['pfx_x'] * 12
savant_df['ABS_HB'] = np.abs(savant_df["HB"])
savant_df['ABS_RelSide'] = np.abs(savant_df["release_pos_x"])

savant_df = savant_df.rename(columns={'player_name': 'Name', "pitch_type" : "Pitch Type", "release_pos_z" : "RelHeight"})

def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

savant_df['Name'] = savant_df["Name"].apply(flip_names)

savant_df['pitcher_team'] = savant_df.apply(lambda row: row['home_team'] if row['inning_topbot'] == 'Top' else row['away_team'], axis=1)

nl_teams = ["ATL", "NYM", "MIA", "PHI", "WSH", "CHC", "CIN", "MIL", "PIT", "STL", "LAD", "ARI", "SFG", "SDP", "COL"]
savant_df["league"] = np.where(savant_df["pitcher_team"].isin(nl_teams), "NL", "AL")


savant_df = savant_df[["league", "pitcher_team", "Name", "Pitch Type", "release_speed", "release_spin_rate", "IVB", "ABS_HB", "RelHeight", "ABS_RelSide", "release_extension"]].dropna()
savant_df = savant_df.reset_index(drop = True).dropna()

savant_df

This is a large query, it may take a moment to complete


100%|██████████| 187/187 [01:13<00:00,  2.55it/s]


,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension
0,AL,BOS,Nick Robertson,CH,89.0,1703,3.96,18.36,5.59,2.8,7.4
1,AL,BOS,Nick Robertson,FF,96.9,2153,16.32,9.12,5.9,2.4,7.4
2,AL,BOS,Nick Robertson,CH,90.0,1698,4.32,19.8,5.56,2.93,7.4
3,AL,BOS,Nick Robertson,SL,82.2,2786,3.36,17.16,5.55,3.09,6.9
4,AL,BOS,Nick Robertson,CH,89.2,1589,3.72,17.88,5.58,2.87,7.2
...,...,...,...,...,...,...,...,...,...,...,...
713249,AL,SEA,Luis Castillo,SI,95.3,2120,5.52,20.28,5.24,3.11,5.6
713250,AL,SEA,Luis Castillo,FF,94.2,2284,13.92,15.36,5.29,3.16,5.6
713251,AL,SEA,Luis Castillo,FF,94.8,2122,14.28,9.6,5.3,3.12,5.4
713252,AL,SEA,Luis Castillo,SI,95.1,2126,4.92,20.52,5.22,3.2,5.5


In [71]:
player = savant_df[savant_df["Name"] == "Corbin Burnes"].reset_index(drop = True).dropna()
player

,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension
0,NL,MIL,Corbin Burnes,FC,94.7,2664,9.12,7.56,5.62,0.92,6.5
1,NL,MIL,Corbin Burnes,FC,95.4,2691,10.92,3.36,5.71,1.0,6.3
2,NL,MIL,Corbin Burnes,SL,83.9,2934,3.12,20.28,5.47,1.07,6.6
3,NL,MIL,Corbin Burnes,SL,88.0,2895,7.08,8.28,5.54,1.04,6.7
4,NL,MIL,Corbin Burnes,FC,94.4,2754,10.68,5.76,5.51,0.99,6.8
...,...,...,...,...,...,...,...,...,...,...,...
3051,NL,MIL,Corbin Burnes,SL,87.9,2991,0.84,6.12,5.87,0.93,6.5
3052,NL,MIL,Corbin Burnes,FC,96.3,2717,17.16,3.96,6.05,0.79,6.3
3053,NL,MIL,Corbin Burnes,SL,89.4,2865,3.0,5.76,5.95,0.79,6.4
3054,NL,MIL,Corbin Burnes,FC,95.2,2617,12.12,1.68,5.9,0.87,6.3


In [72]:
team = savant_df[savant_df["pitcher_team"] == "ATL"].reset_index(drop = True).dropna()
team

,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension
0,NL,ATL,Michael Tonkin,SL,82.0,2195,-5.52,2.52,5.4,3.83,6.3
1,NL,ATL,Michael Tonkin,SI,94.6,2454,14.16,9.12,5.45,3.63,6.2
2,NL,ATL,Michael Tonkin,SI,94.1,2495,10.92,9.24,5.36,3.8,6.3
3,NL,ATL,Michael Tonkin,SL,84.0,2452,-2.16,3.36,5.38,3.73,6.2
4,NL,ATL,Michael Tonkin,SL,83.9,2353,0.24,3.36,5.46,3.65,6.2
...,...,...,...,...,...,...,...,...,...,...,...
23927,NL,ATL,Max Fried,CU,76.6,2692,-15.84,8.28,6.23,1.29,6.3
23928,NL,ATL,Max Fried,CU,75.6,2866,-16.44,5.64,6.19,1.23,6.5
23929,NL,ATL,Max Fried,CH,87.3,1701,5.16,17.76,5.88,1.51,6.4
23930,NL,ATL,Max Fried,CU,73.1,2612,-13.2,6.72,6.29,1.29,6.0


In [73]:
league = savant_df[savant_df["league"] == "NL"].reset_index(drop = True).dropna()
league

,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension
0,NL,MIL,Janson Junk,CU,83.6,2444,-13.68,6.48,5.41,1.21,6.7
1,NL,MIL,Janson Junk,FF,95.7,2299,15.72,4.8,5.43,1.3,6.6
2,NL,MIL,Janson Junk,FF,93.9,2357,19.68,2.16,5.59,1.14,6.6
3,NL,MIL,Janson Junk,FF,93.8,2358,19.08,1.32,5.69,1.14,6.5
4,NL,MIL,Janson Junk,FF,94.7,2296,19.08,2.64,5.56,1.06,6.6
...,...,...,...,...,...,...,...,...,...,...,...
285491,NL,LAD,Julio Urías,FF,92.5,2549,18.24,0.36,6.05,1.48,5.4
285492,NL,LAD,Julio Urías,SV,79.2,2929,-7.32,18.0,6.06,1.24,5.4
285493,NL,LAD,Julio Urías,FC,86.3,2775,3.48,7.92,6.08,1.2,5.7
285494,NL,LAD,Julio Urías,FF,92.6,2574,19.92,2.76,6.18,1.24,5.4


# For Players

In [74]:
stuff = player.drop(["league", "pitcher_team", "Name", "Pitch Type"], axis = 1)

In [94]:
for i in range(len(player)):
    pitch_type = player.loc[i, 'Pitch Type']
    
    if pitch_type == "SI":
        player.at[i, "Stuff"] = SIrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sinker["Whiff"] == 1) / len(df_sinker["Whiff"])) * 100
  
    elif pitch_type == "FC":
        player.at[i, "Stuff"] = CTrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_cutter["Whiff"] == 1) / len(df_cutter["Whiff"])) * 100

    elif pitch_type == "FF":
        player.at[i, "Stuff"] = FBrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_fastball["Whiff"] == 1) / len(df_fastball["Whiff"])) * 100

    elif pitch_type == "CH":
        player.at[i, "Stuff"] = CHrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_changeup["Whiff"] == 1) / len(df_changeup["Whiff"])) * 100

    elif pitch_type == "SP" or pitch_type == "FO"  or pitch_type == "FS":
        player.at[i, "Stuff"] = SPrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_splitter["Whiff"] == 1) / len(df_splitter["Whiff"])) * 100

    elif pitch_type == "CU" or pitch_type == "KC":
        player.at[i, "Stuff"] = CUrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_curveball["Whiff"] == 1) / len(df_curveball["Whiff"])) * 100

    elif pitch_type == "SL" or pitch_type == "SV":
        player.at[i, "Stuff"] = SLrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_slider["Whiff"] == 1) / len(df_slider["Whiff"])) * 100

    elif pitch_type == "ST":
        player.at[i, "Stuff"] = STrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sweeper["Whiff"] == 1) / len(df_sweeper["Whiff"])) * 100

In [95]:
player_grouped = player.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2)

player_grouped

Pitches   Velo  Max_Velo  SpinRate    IVB  ABS_HB  \
Name          Pitch Type                                                      
Corbin Burnes CH              344  89.32      91.9   2048.71   7.07   13.83   
              CU              526  79.86      83.7   2794.21 -10.94   10.35   
              FC             1693  94.43      97.3   2655.19  11.24     3.5   
              SI              235  95.31      97.8   2489.25  10.89   10.68   
              SL              258  86.03      91.8   2876.97   1.56   12.45   

                          Release_Height  Extension   Stuff  
Name          Pitch Type                                     
Corbin Burnes CH                    5.69       6.62  118.55  
              CU                    6.07       6.53  157.21  
              FC                    5.91       6.57  144.38  
              SI                    5.88       6.62  192.40  
              SL                    5.79       6.56  114.94

# For Teams

In [77]:
stuff = team.drop(["league", "pitcher_team", "Name", "Pitch Type"], axis = 1)

In [78]:
for i in range(len(team)):
    pitch_type = team.loc[i, 'Pitch Type']
    
    if pitch_type == "SI":
        team.at[i, "Stuff"] = SIrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sinker["Whiff"] == 1) / len(df_sinker["Whiff"])) * 100
  
    elif pitch_type == "FC":
        team.at[i, "Stuff"] = CTrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_cutter["Whiff"] == 1) / len(df_cutter["Whiff"])) * 100

    elif pitch_type == "FF":
        team.at[i, "Stuff"] = FBrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_fastball["Whiff"] == 1) / len(df_fastball["Whiff"])) * 100

    elif pitch_type == "CH":
        team.at[i, "Stuff"] = CHrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_changeup["Whiff"] == 1) / len(df_changeup["Whiff"])) * 100

    elif pitch_type == "SP" or pitch_type == "FO" or pitch_type == "FS":
        team.at[i, "Stuff"] = SPrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_splitter["Whiff"] == 1) / len(df_splitter["Whiff"])) * 100

    elif pitch_type == "CU" or pitch_type == "KC":
        team.at[i, "Stuff"] = CUrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_curveball["Whiff"] == 1) / len(df_curveball["Whiff"])) * 100

    elif pitch_type == "SL" or pitch_type == "SV":
        team.at[i, "Stuff"] = SLrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_slider["Whiff"] == 1) / len(df_slider["Whiff"])) * 100

    elif pitch_type == "ST":
        team.at[i, "Stuff"] = STrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sweeper["Whiff"] == 1) / len(df_sweeper["Whiff"])) * 100

In [79]:
team_grouped = team.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index()

team_grouped

,Name,Pitch Type,Pitches,Velo,Max_Velo,SpinRate,IVB,ABS_HB,Release_Height,Extension,Stuff
0,A.J. Minter,CH,144,86.68,89.2,1636.7,5.05,15.39,5.66,6.23,91.70
1,A.J. Minter,FC,408,89.45,92.2,2557.94,3.01,5.44,5.8,6.28,155.44
2,A.J. Minter,FF,496,95.84,97.7,2426.04,16.7,6.51,5.79,6.38,137.95
3,AJ Smith-Shawver,CH,48,86.09,89.0,1354.75,9.64,9.78,6.0,6.7,104.59
4,AJ Smith-Shawver,CU,55,76.79,80.3,2255.8,-13.02,8.31,6.23,6.53,104.90
...,...,...,...,...,...,...,...,...,...,...,...
117,Taylor Hearn,SL,3,88.33,89.6,2032.67,2.88,3.72,6.61,6.27,91.73
118,Yonny Chirinos,FF,12,93.08,94.2,2188.83,13.9,9.64,6.53,6.16,67.31
119,Yonny Chirinos,FS,151,85.72,88.1,1356.2,3.63,6.08,6.81,6.35,115.05
120,Yonny Chirinos,SI,180,92.73,95.6,2037.03,6.77,16.07,6.61,6.33,208.29


In [80]:
Sinker = team_grouped[team_grouped["Pitch Type"] == "SI"].sort_values(by='Stuff', ascending = False)

In [81]:
Cutter = team_grouped[team_grouped["Pitch Type"] == "FC"].sort_values(by='Stuff', ascending = False)

In [82]:
Fastball = team_grouped[team_grouped["Pitch Type"] == "FF"].sort_values(by='Stuff', ascending = False)

In [83]:
Changeup = team_grouped[team_grouped["Pitch Type"] == "CH"].sort_values(by='Stuff', ascending = False)

In [84]:
Splitter = team_grouped[team_grouped["Pitch Type"].isin(["SP", "FO", "FS"])].sort_values(by='Stuff', ascending = False)

In [85]:
Curveball = team_grouped[team_grouped["Pitch Type"].isin(["CU", "KC"])].sort_values(by='Stuff', ascending = False)

In [86]:
Slider = team_grouped[team_grouped["Pitch Type"].isin(["SL", "SV"])].sort_values(by='Stuff', ascending = False)

In [87]:
Sweeper = team_grouped[team_grouped["Pitch Type"] == "ST"].sort_values(by='Stuff', ascending = False)

In [88]:
overall = team.groupby(['Name',]).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index().sort_values(by='Stuff', ascending = False)

overall

,Name,Pitches,Velo,Max_Velo,SpinRate,IVB,ABS_HB,Release_Height,Extension,Stuff
8,Danny Young,135,83.63,90.4,2537.7,3.07,19.36,5.43,5.63,157.12
31,Yonny Chirinos,394,89.74,95.6,1803.25,5.83,10.29,6.69,6.33,157.07
30,Taylor Hearn,24,95.3,97.3,2281.25,13.75,8.74,6.6,6.32,141.60
17,Joe Jiménez,936,91.53,97.5,2334.18,11.82,8.37,5.83,7.11,139.84
29,Spencer Strider,3070,92.57,100.5,2348.54,12.18,6.54,5.85,7.05,138.64
0,A.J. Minter,1048,92.1,97.7,2368.93,9.77,7.31,5.77,6.32,138.40
27,Pierce Johnson,341,88.33,97.4,2789.66,0.57,8.78,6.11,5.87,132.09
24,Michael Tonkin,1281,89.43,96.0,2290.42,5.44,9.08,5.32,6.26,127.75
6,Charlie Morton,2840,88.05,98.0,2629.09,1.02,13.22,5.37,6.15,125.59
18,Kirby Yates,1058,90.02,95.9,1882.28,7.09,13.15,5.32,6.15,122.91


# For League

In [89]:
stuff = league.drop(["league", "pitcher_team", "Name", "Pitch Type"], axis = 1)

In [96]:
for i in range(len(league)):
    pitch_type = league.loc[i, 'Pitch Type']
    
    if pitch_type == "SI":
        league.at[i, "Stuff"] = SIrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sinker["Whiff"] == 1) / len(df_sinker["Whiff"])) * 100
  
    elif pitch_type == "FC":
        league.at[i, "Stuff"] = CTrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_cutter["Whiff"] == 1) / len(df_cutter["Whiff"])) * 100

    elif pitch_type == "FF":
        league.at[i, "Stuff"] = FBrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_fastball["Whiff"] == 1) / len(df_fastball["Whiff"])) * 100

    elif pitch_type == "CH":
        league.at[i, "Stuff"] = CHrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_changeup["Whiff"] == 1) / len(df_changeup["Whiff"])) * 100

    elif pitch_type == "SP" or pitch_type == "FO" or pitch_type == "FS":
        league.at[i, "Stuff"] = SPrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_splitter["Whiff"] == 1) / len(df_splitter["Whiff"])) * 100

    elif pitch_type == "CU" or pitch_type == "KC":
        league.at[i, "Stuff"] = CUrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_curveball["Whiff"] == 1) / len(df_curveball["Whiff"])) * 100

    elif pitch_type == "SL" or pitch_type == "SV":
        league.at[i, "Stuff"] = SLrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_slider["Whiff"] == 1) / len(df_slider["Whiff"])) * 100

    elif pitch_type == "ST":
        league.at[i, "Stuff"] = STrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sweeper["Whiff"] == 1) / len(df_sweeper["Whiff"])) * 100

In [99]:
overall = league.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index().sort_values(by='Stuff', ascending = False)

overall = overall[overall['Pitches'] >= 100]
overall

,Name,Pitch Type,Pitches,Velo,Max_Velo,SpinRate,IVB,ABS_HB,Release_Height,Extension,Stuff
863,Jose Cuas,FF,101,92.43,94.5,2395.71,9.67,11.53,4.04,6.38,210.77
1509,Yonny Chirinos,SI,180,92.73,95.6,2037.03,6.77,16.07,6.61,6.33,208.29
528,Elvis Peguero,SI,447,95.98,99.4,1855.39,1.91,16.57,5.36,6.95,207.94
17,Abner Uribe,SI,317,99.37,101.9,2173.33,6.37,13.92,5.99,6.49,202.17
1377,Tanner Scott,FF,570,96.81,100.5,2598.13,15.58,4.09,5.17,6.38,199.99
...,...,...,...,...,...,...,...,...,...,...,...
418,Darius Vines,FF,102,89.8,92.1,2157.08,16.78,5.64,6.09,6.37,66.87
173,Brad Hand,FF,322,92.62,94.9,2307.8,12.19,11.21,6.11,5.91,65.33
221,Bryce Elder,FF,378,90.72,93.0,2124.07,14.98,4.0,6.53,6.07,64.06
105,Andrew Suárez,FF,236,92.34,94.7,1895.03,14.53,7.81,6.3,5.77,63.89


In [108]:
overall = league.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index().sort_values(by='Stuff', ascending = False)

overall = overall[overall['Pitches'] >= 600]
overall

,Name,Pitch Type,Pitches,Velo,Max_Velo,SpinRate,IVB,ABS_HB,Release_Height,Extension,Stuff
558,Eury Pérez,FF,697,97.53,100.1,2634.53,18.12,8.97,6.06,6.86,165.85
1525,Zack Wheeler,FF,1366,95.79,98.5,2494.92,15.3,6.29,5.44,7.25,154.48
284,Charlie Morton,CU,1226,82.33,85.2,3088.32,-7.83,13.6,5.33,6.11,153.94
479,Drew Smith,FF,663,95.0,97.9,2503.17,18.13,7.51,5.54,6.12,153.31
578,Freddy Peralta,FF,1453,94.44,98.7,2414.19,15.46,6.89,5.08,6.91,152.21
...,...,...,...,...,...,...,...,...,...,...,...
86,Andre Pallante,FF,739,96.06,98.7,2235.29,8.88,3.07,5.96,6.48,80.93
139,Austin Gomber,FF,947,90.59,93.8,2169.77,17.11,7.08,6.67,6.63,79.57
1254,Rich Hill,FF,666,88.2,90.9,2203.23,17.34,6.65,5.93,6.31,79.53
1252,Rich Hill,CU,746,71.78,75.4,2617.16,-12.68,16.7,5.73,6.06,79.20


In [97]:
overall = league.groupby(['Name']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index().sort_values(by='Stuff', ascending = False)

overall = overall[overall['Pitches'] >= 100]
overall

,Name,Pitches,Velo,Max_Velo,SpinRate,IVB,ABS_HB,Release_Height,Extension,Stuff
211,José Alvarado,742,96.37,101.6,2145.07,11.51,8.43,6.68,6.88,196.36
4,Abner Uribe,518,96.23,103.3,2331.36,5.35,11.69,6.0,6.49,174.04
124,Elvis Peguero,912,93.25,99.4,2182.68,1.04,9.49,5.4,6.9,169.81
334,Tanner Scott,1205,92.92,100.5,2669.33,7.01,5.36,5.08,6.4,169.80
152,Hoby Milner,1000,84.26,91.3,1882.52,1.35,16.08,4.27,6.83,165.23
...,...,...,...,...,...,...,...,...,...,...
107,Devin Smeltzer,361,83.48,91.6,2087.88,5.28,12.19,5.91,5.8,85.89
26,Andrew Suárez,520,86.97,94.7,2046.97,7.69,8.42,6.28,5.75,82.18
48,Brett Kennedy,309,89.65,95.6,1918.17,9.83,11.99,5.73,6.46,81.11
42,Brad Hand,968,86.83,95.1,2333.35,5.41,13.48,6.1,5.8,79.51


In [110]:
overall = league.groupby(['Name']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index().sort_values(by='Stuff', ascending = False)

overall = overall[overall['Pitches'] >= 1300]
overall

,Name,Pitches,Velo,Max_Velo,SpinRate,IVB,ABS_HB,Release_Height,Extension,Stuff
131,Eury Pérez,1533,91.06,100.1,2442.36,11.22,6.46,5.94,6.84,146.42
372,Zack Wheeler,3146,91.91,98.5,2515.75,8.1,8.86,5.4,7.17,143.60
40,Bobby Miller,1958,92.22,101.5,2441.81,6.52,10.5,5.98,6.89,141.31
327,Spencer Strider,3070,92.57,100.5,2348.54,12.18,6.54,5.85,7.05,138.64
136,Freddy Peralta,2839,88.79,98.7,2265.54,8.56,7.77,5.07,6.92,136.46
...,...,...,...,...,...,...,...,...,...,...
44,Brandon Williamson,1937,87.92,96.0,2212.04,8.68,8.8,6.53,5.96,94.14
291,Peter Lambert,1419,89.07,95.8,2116.02,8.85,8.61,5.94,6.38,92.87
301,Rich Hill,2011,79.55,90.9,2384.49,2.33,11.5,5.63,6.19,92.32
275,Miles Mikolas,3184,87.52,96.6,2278.7,4.9,8.15,6.55,6.4,91.10
